## Limpieza de Datos: Compras Públicas Municipales

Este notebook documenta el proceso de limpieza y estandarización de datos de compras públicas municipales en Chile, obtenidos de la plataforma Mercado Público. El objetivo es preparar los datasets para un EDA posterior.

Para correr este notebook es necesario tener un una carpeta llamada Municipalidad con los datos en formato csv, en el mismo directorio.

Se trabajará con 4 datasest: 

*  Compra Ágil
* Convenio Marco
* Licitación
* Trato Directo




In [1]:
import pandas as pd

In [ ]:
'''
Aqui voy a seleccionar algunas columnas de interés

'''

columns_to_keep = [
    "codigoOC",
    "NroLicitacion",
    "FechaEnvioOC",
    "EstadoOC",
    "NombreOC", 

    # Institución compradora
    "Institucion",
    "UnidadCompra",
    "RegionUnidadCompra",
    "Sector",

    # Proveedor
    "Proveedor",
    "ProveedorRUT",
    "TamanoProveedor",
    "RegionProveedor",

    # Montos y compra
    "MontoNetoOC_CLP",
    "MontoTotalOC",
    "ProcedenciaOC",

    # Producto / Servicio
    "NombreItem",
    "CantidadItem",
    "MontoTotalItem",
    "RubroN1",
    "RubroN2",
    "RubroN3",
    "ONUProducto"
]

df = pd.read_csv("Municipalidades/Licitacion.csv", sep=";", encoding="latin1", usecols=columns_to_keep)

In [ ]:
'''
Limpieza básica de eliminación de nulos y duplicados
'''

df = df.dropna(subset=["Proveedor", "MontoTotalOC"])
df = df.drop_duplicates()

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 234748 entries, 0 to 250821
Data columns (total 23 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   codigoOC            234748 non-null  object
 1   NroLicitacion       234748 non-null  object
 2   FechaEnvioOC        234748 non-null  object
 3   NombreOC            234748 non-null  object
 4   EstadoOC            234748 non-null  object
 5   ProcedenciaOC       234748 non-null  object
 6   MontoTotalOC        234748 non-null  object
 7   MontoNetoOC_CLP     234707 non-null  object
 8   UnidadCompra        234748 non-null  object
 9   RegionUnidadCompra  233892 non-null  object
 10  Institucion         234748 non-null  object
 11  Sector              234748 non-null  object
 12  Proveedor           234748 non-null  object
 13  ProveedorRUT        234748 non-null  object
 14  TamanoProveedor     234748 non-null  object
 15  RegionProveedor     229954 non-null  object
 16  RubroN1